## Section 3: Mechansim of induction heads and common subspace propoerty
### This notebook only looks at GPT-2 

**The overall plan is to provide evidence for the four main arguments**.

1. Multiplying weight matrix W_QK and W_OV yields large diagonal values
   
2. The top right singular subspace of W_QK matches the top left singular subspaec of W_OV
   
3. (Invariance) Shuffling induction heads do not significantly impact performance of copying

4. (Common subspace as composition pathway) Project out common subspace in induction heads disables copying, while only keeping common subspace is sufficient for copying

In [1]:
import torch
import numpy as np
import os
import math
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import seaborn as sns
from torch.nn import functional as F

from transformers import logging
from transformers import set_seed
from transformers import GPT2Model, GPT2Config
from transformers import GPT2Tokenizer

from analysis_utils import create_folder, qkov_matching, subspace_matching

set_seed(2024)

### NOTE: run Step 1-1-1, 1-1-2, 1-1-3, 1-1-4, 1-1-5 to get started. 

### Calulating embeddings and matrices

### Step 1-1-1: loading GPT-2 and setting global variables

In [2]:
configuration = GPT2Config()
model = GPT2Model.from_pretrained("gpt2", output_attentions=True)
configuration = model.config
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab_size = 50257
T0 = 5

sample_int = (
    np.random.randint(low=0, high=vocab_size, size=T0)
    .repeat(3)
    .reshape(T0, -1)
    .T.ravel()
)
input_ids = torch.Tensor(sample_int).long().unsqueeze(0)

with torch.no_grad():
    output = model(
        input_ids
    )  # (num_layer, num_head, seq_length, seq_length) = (12, 12, 15, 15)
len(output.attentions), output.attentions[0].size()

attentions = np.array([a.detach().numpy()[0] for a in output.attentions])
attentions.shape

(12, 12, 15, 15)

### Step 1-1-2: Calculating embeddings for each sublayer

In [3]:
num_layer = 12
num_heads = 12
d_model = 768
d_head = d_model // num_heads
seq_len = input_ids.size(1)

model.eval()

hiddens_all = torch.zeros(num_layer + 1, 6, seq_len, d_model)

h = model.wte(input_ids)
pos = torch.arange(0, seq_len, dtype=torch.long).unsqueeze(0)
h = model.drop(model.wte(input_ids) + model.wpe(pos))
hiddens_all[0] = h.squeeze()
for layer in range(num_layer):
    h2 = model.h[layer].ln_1(h)  # LayerNorm_1 output embeddings, (seq_length, d_model)
    h4 = model.h[layer].attn(h2)[
        0
    ]  # Self-attention output embeddings (seq_length, d_model)
    h5 = h + h4  # Adding with identity component, (seq_length, d_model)
    h6 = model.h[layer].ln_2(h5)  # LayerNorm_2 output embeddings, (seq_length, d_model)
    h = model.h[layer](h)[0]  # Next-layer embeddings, (seq_length, d_model)

    hiddens_all[layer + 1, 0] = h.squeeze()
    hiddens_all[layer, 1] = h2.squeeze()
    # hiddens_all[layer, 2] = torch.tensor(attentions[layer,layer,:,:]) @ h2.squeeze()  # NOT CORRECT
    hiddens_all[layer, 3] = h4.squeeze()
    hiddens_all[layer, 4] = h5.squeeze()
    hiddens_all[layer, 5] = h6.squeeze()


def cosine_sim(x, y, prec_digit=4):
    x, y = torch.tensor(x), torch.tensor(y)
    out = torch.sum(x * y) / (torch.norm(x) * torch.norm(y))
    return np.around(out.numpy(force=True), decimals=prec_digit)


def matrix_mask(d, return_one=True, offsets=[0]):
    offsets = [torch.tensor(offset) for offset in offsets]
    mask = torch.zeros(d, d, dtype=torch.bool)
    for offset in offsets:
        mask += torch.diag(torch.ones(d - torch.abs(offset), dtype=torch.bool), offset)
    mask = mask if return_one else ~mask
    return mask

### Step 1-1-3: Calculating QK and OV matrices for all layers

In [4]:
W0 = model.wte.weight
vals_flip, vecs_flip = np.linalg.eigh((W0.T @ W0).numpy(force=True))
vals, vecs = vals_flip[::-1], vecs_flip[:, ::-1]

W_all = torch.zeros(num_layer, num_heads, 4, d_model, d_head)

for layer in range(num_layer):
    W_q, W_k, W_v = model.h[layer].attn.c_attn.weight.split(d_model, dim=1)
    W_q = W_q.view(d_model, num_heads, d_model // num_heads)
    W_k = W_k.view(d_model, num_heads, d_model // num_heads)
    W_v = W_v.view(d_model, num_heads, d_model // num_heads)
    W_o = model.h[layer].attn.c_proj.weight.view(
        num_heads, d_model // num_heads, d_model
    )

    for head in range(num_heads):
        W_all[layer, head, 0] = W_q[:, head, :]  # (d_model, d_head)
        W_all[layer, head, 1] = W_k[:, head, :]  # (d_model, d_head)
        W_all[layer, head, 2] = W_v[:, head, :]  # (d_model, d_head)
        W_all[layer, head, 3] = W_o[head, :, :].T  # (d_model, d_head)

### Step 1-1-4: Measuring induction head (ranking most likely induction head to most unlikely)

In [5]:
dir_name = "simple_visz"
create_folder(dir_name)
filename = os.path.join(dir_name, f"induction_head_scores.txt")

scores = np.zeros((num_layer, num_heads))
for layer in range(num_layer):
    for head in range(num_heads):
        A = attentions[layer, head]
        A_adjusted = np.zeros((seq_len, seq_len))
        A_adjusted[1:, 1:] = A[1:, 1:] / np.sum(A[1:, 1:], axis=1, keepdims=True)
        diag1 = np.diag(A_adjusted, -(T0 - 1))[1:]
        diag2 = np.diag(A_adjusted, -(2 * T0 - 1))[1:]
        diag = np.concatenate((diag1[:-T0], diag1[-T0:] + diag2))
        scores[layer, head] = np.mean(diag)

idx_sort = np.argsort(scores, axis=None)[::-1]
IH_list = [
    [idx_sort[j] // num_heads, idx_sort[j] % num_heads] for j in range(len(idx_sort))
]

with open(filename, "w") as file:
    print(
        f"Ranking induction heads (most likely to unlikely) by attention scores",
        file=file,
    )
    for j, pair in enumerate(IH_list):
        layer, head = pair[0], pair[1]
        print(f"Layer {layer} Head {head}: score {scores[layer, head]}", file=file)
        if j <= 20:
            print(f"Layer {layer} Head {head}: score {scores[layer, head]}")

Layer 5 Head 1: score 0.9953717850148678
Layer 5 Head 5: score 0.9902818202972412
Layer 6 Head 9: score 0.9886472046375274
Layer 7 Head 10: score 0.9395123027265072
Layer 5 Head 8: score 0.921284668892622
Layer 10 Head 7: score 0.9204489454627037
Layer 5 Head 0: score 0.88764777071774
Layer 7 Head 2: score 0.8102712273597718
Layer 10 Head 1: score 0.805915080010891
Layer 9 Head 9: score 0.7934778049588204
Layer 9 Head 6: score 0.7573517814278603
Layer 11 Head 10: score 0.7551813259720802
Layer 6 Head 1: score 0.6811699584126473
Layer 10 Head 6: score 0.6629791051149369
Layer 8 Head 1: score 0.6548118874430656
Layer 10 Head 0: score 0.6461947090923786
Layer 7 Head 11: score 0.6174159724265337
Layer 8 Head 6: score 0.5985757991671562
Layer 10 Head 10: score 0.5727812275290489
Layer 8 Head 11: score 0.5598440110683441
Layer 9 Head 1: score 0.5226531833410263


### Step 1-1-5: Measuring Shifting Head (ranking most likely induction head to most unlikely)

In [6]:
dir_name = "simple_visz"
create_folder(dir_name)
filename = os.path.join(dir_name, f"shifting_head_scores.txt")

scores = np.zeros((num_layer, num_heads))
for layer in range(num_layer):
    for head in range(num_heads):
        A = attentions[layer, head]
        A_adjusted = np.zeros((seq_len, seq_len))
        A_adjusted[1:, 1:] = A[1:, 1:] / np.sum(A[1:, 1:], axis=1, keepdims=True)
        diag = np.diag(A_adjusted, -1)[1:]
        scores[layer, head] = np.mean(diag)

idx_sort = np.argsort(scores, axis=None)[::-1]
Shifting_list = [
    [idx_sort[j] // num_heads, idx_sort[j] % num_heads] for j in range(len(idx_sort))
]

with open(filename, "w") as file:
    print(
        f"Ranking shifting heads (most likely to unlikely) by attention scores",
        file=file,
    )
    for j, pair in enumerate(Shifting_list):
        layer, head = pair[0], pair[1]
        print(f"Layer {layer} Head {head}: score {scores[layer, head]}", file=file)
        if j <= 20:
            print(f"Layer {layer} Head {head}: score {scores[layer, head]}")

Layer 4 Head 11: score 0.9993197276042058
Layer 5 Head 6: score 0.7670075434904832
Layer 6 Head 8: score 0.7444457411766052
Layer 3 Head 7: score 0.640053208057697
Layer 3 Head 3: score 0.5796716167376592
Layer 2 Head 2: score 0.5276686915984521
Layer 7 Head 0: score 0.5010626384845147
Layer 2 Head 5: score 0.48505993760549104
Layer 3 Head 2: score 0.43790112837002826
Layer 3 Head 8: score 0.4295523739778079
Layer 4 Head 3: score 0.41924109481848204
Layer 5 Head 4: score 0.41646881516163164
Layer 8 Head 7: score 0.411526409479288
Layer 2 Head 9: score 0.4076269922348169
Layer 3 Head 6: score 0.3934386917031728
Layer 2 Head 8: score 0.3858961043162988
Layer 4 Head 6: score 0.3732431691426497
Layer 7 Head 8: score 0.36464981505504024
Layer 2 Head 3: score 0.35175169431246245
Layer 6 Head 0: score 0.3336490748020319
Layer 5 Head 2: score 0.3260213073629599


## Below we provide evidence for our claims

### Claim 1: large diagonal values in W_qkov

**We plan to select a few representative heatmaps of W_qkov to show.**

**Further, we plan to calculate and perhaps show some statistics**

1. qkov_matching_summary calculates z-scores of every top (IH, Shifting head) pair, and plots a heatmap. A value bigger than 2 suggests that diagonal line is significantly larger than other off-diagonal values.

2. We need to pay attention to how to choose the thresholds for determining "top" IH and Shifting heads. Here for simplicity, I just choose top-10 heads. Perhaps there are more clear induction heads and fewer clear shifting heads.


In [ ]:
create_folder("Figs")
create_folder("Figs/diagonal")

res = qkov_matching(W_all, IH_list[:10], "Figs/diagonal")

In [ ]:
from analysis_utils import qkov_matching_summary

K0, K1 = 20, 20
scores = qkov_matching_summary(W_all, IH_list[:K0], Shifting_list[:K1], "Figs/diagonal")

### Claim 2: subspace matching

**Generalized cosine similarity**: to measure how similar two subspaces are, I use two methods named "largest" and "mean". Both methods give values between 0 (orthogonal) to 1 (aligned).

- "largest" is a favorable score that finds the best vectors in each subspaces to maximize the regular cosine similarity
- "mean" reflects how similar a random vector in a subspace is to a random vector in another subspace

**Three matching measurements**: Among induction heads, among shifting heads, and between IH and shifting

**Finding**
- Under "largest", many subspaces match well (score > 0.8)
- Under "mean", many subspaces are positively correlated (0.2--0.6). It suggests subspaces are not perfectly aligned, but there are nontrivial correlation and much better than random subspaces (which are almost orthogonal, values << 0.1)

In [ ]:
K = 10
s_match1, match_baseline1 = subspace_matching(
    W_all, "Figs/subspace_matching_IH", IH_list=IH_list[:K], ranks=[2, 3, 5, 10]
)
s_match2, match_baseline2 = subspace_matching(
    W_all,
    "Figs/subspace_matching_SH",
    Shifting_list=Shifting_list[:K],
    ranks=[2, 3, 5, 10],
)
s_match3, match_baseline3 = subspace_matching(
    W_all,
    "Figs/subspace_matching_IH_SH",
    IH_list=IH_list[:K],
    Shifting_list=Shifting_list[:K],
    ranks=[2, 3, 5, 10],
)

### Claim 3: invariance under shuffling

**Error reporting for edited models**. I used a batch of repeated random tokens to measure how well an edited model performs copying.


**Ideal plots**. We hope to see prediction errors remain low under random shuffling, and random baseline has high errors. As we increase more heads to shuffle, prediction errors increase.

**Issues**.

1. **High variability**. Because each edited model only use one random permutation, its performance depends on how several important heads are permuted. To draw conclusions, we should run many independent experiments. Below for conveience I use run one experiment for each K.

2. **Results about shifting heads are weird**. Is it due to variability? Is there a bug? When I increase K, errors of original, edited, random baseline are similar.

In [ ]:
from analysis_utils import exchange_edit, pred_probs, shuffle_exp

seg_len = 20  # repeating segment length
for K in tqdm(range(5, 21)):
    K0, K1 = K, K
    save_dir = "Figs/shuffle"
    probs_QK, errs_QK = shuffle_exp(
        model,
        configuration,
        seg_len,
        vocab_size,
        IH_list[:K0],
        save_dir,
        component="QK",
    )
    probs_OV, errs_OV = shuffle_exp(
        model,
        configuration,
        seg_len,
        vocab_size,
        Shifting_list[:K1],
        save_dir,
        component="OV",
    )

### Claim 4: projecting weight matrix

- Experiments on induction heads suggest common subspace of rank ~ 50 is crucial for copying.

- Experiments on shifting heads are bad. I don't know if there is a bug.

In [7]:
from analysis_utils import project_exp

save_dir = "Figs/project_test"
seg_len = 20
probs, errs = project_exp(
    model,
    configuration,
    W_all,
    seg_len,
    vocab_size,
    IH_list[:10],
    IH_list[:30],
    save_dir,
    component="QK",
    project_out=True,
    max_rank=600,
    step=20,
)

100%|██████████| 30/30 [01:32<00:00,  3.07s/it]


In [ ]:
probs, errs = project_exp(
    model,
    configuration,
    W_all,
    seg_len,
    vocab_size,
    IH_list[:10],
    IH_list[:30],
    save_dir,
    component="QK",
    project_out=False,
)

In [ ]:
from analysis_utils import project_exp

save_dir = "Figs/project"
seg_len = 20

probs, errs = project_exp(
    model,
    configuration,
    W_all,
    seg_len,
    vocab_size,
    IH_list[:10],
    Shifting_list[:30],
    save_dir,
    component="OV",
    project_out=True,
    max_rank=100,
    step=5,
)
probs, errs = project_exp(
    model,
    configuration,
    W_all,
    seg_len,
    vocab_size,
    IH_list[:10],
    Shifting_list[:30],
    save_dir,
    component="OV",
    project_out=False,
    max_rank=100,
    step=5,
)